<a href="https://colab.research.google.com/github/knsg16/Yatagarasu-Eye/blob/main/yatagarasu_eye.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# JSONデータの解析

In [1]:
import json
from google.colab import drive

# google driveのマウント
drive.mount('/content/drive')

# JSONファイルの読み込み
with open('/content/drive/MyDrive/Yatagarasu-Eye/annotations/instances_default.json', 'r') as file:
    data = json.load(file)

# 画像ファイル名とラベルの抽出
image_labels = {img['file_name']: any(ann['image_id'] == img['id'] and ann['category_id'] == 1 for ann in data['annotations']) for img in data['images']}


Mounted at /content/drive


# 画像データの前処理

In [2]:
import cv2
import numpy as np
import os
from concurrent.futures import ThreadPoolExecutor

base_path = '/content/drive/MyDrive/Yatagarasu-Eye/images'

def load_and_preprocess_image(file_name):
    img_path = os.path.join(base_path, file_name)
    img = cv2.imread(img_path)
    img = cv2.resize(img, (1280, 720))
    img = img / 255.0
    return img

# 並列処理を使用して画像を前処理
with ThreadPoolExecutor(max_workers=10) as executor:
    preprocessed_images = {file_name: executor.submit(load_and_preprocess_image, file_name) for file_name in image_labels.keys()}

# 結果を収集
preprocessed_images = {file_name: future.result() for file_name, future in preprocessed_images.items()}


# シーケンスデータの作成

In [3]:
sequence_length = 30  # 連続するフレーム数
sequences = []
labels = []

file_names = list(preprocessed_images.keys())
for i in range(len(file_names) - sequence_length):
    sequence = [preprocessed_images[file_names[j]] for j in range(i, i + sequence_length)]
    label = image_labels[file_names[i + sequence_length - 1]]
    sequences.append(sequence)
    labels.append(label)


# トレーニングデータの準備

In [16]:
import tensorflow as tf


def data_generator():
    for seq, label in zip(sequences, labels):
        yield seq, label

# データセットの作成
output_signature = (
    tf.TensorSpec(shape=(30, 720, 1280, 3), dtype=tf.float32),  # バッチ次元を除去
    tf.TensorSpec(shape=(), dtype=tf.int32),  # バッチ次元を除去
)
dataset = tf.data.Dataset.from_generator(data_generator, output_signature=output_signature)

# バッチサイズを8に設定
dataset = dataset.batch(8)

# モデルの作成

In [11]:
import tensorflow as tf
from tensorflow.keras import layers, models

model = models.Sequential([
    layers.Conv3D(8, kernel_size=(3, 3, 3), activation='relu', input_shape=(30, 720, 1280, 3)),
    layers.MaxPooling3D(pool_size=(2, 2, 2)),
    layers.Conv3D(16, kernel_size=(3, 3, 3), activation='relu'),
    layers.MaxPooling3D(pool_size=(2, 2, 2)),
    # 他のConv3DレイヤーやMaxPooling3Dレイヤーを追加
    layers.Flatten(),
    layers.Dense(32, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

# モデルのトレーニング

In [12]:
for x_batch, y_batch in dataset.take(1):
    # このバッチの形状を印刷
    print('x_batch shape:', x_batch.shape)
    print('y_batch shape:', y_batch.shape)



x_batch shape: (8, 30, 720, 1280, 3)
y_batch shape: (8,)


In [17]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(dataset, epochs=20)


Epoch 1/20


ResourceExhaustedError: ignored